In [1]:
import sys
import os
import pandas as pd
import tensorflow as tf
from datetime import datetime

# Add path to Helpers to import model_utils
sys.path.append(os.path.abspath('../Helpers'))
import model_utils as mu

In [2]:
# --- 1. Load Data ---
BASE_PATH = '../data_recordings'
print(f"Loading manifest from: {BASE_PATH}")
df = mu.load_manifest(BASE_PATH)

# Optional: Sample data for quick debugging (uncomment next line)
# df = df.head(50)

Loading manifest from: ../data_recordings


In [3]:
# Load and process audio into spectrograms
X, y, speakers = mu.load_and_process_data(df, BASE_PATH)

Loading 11225 audio files...
Successfully loaded: 11225, Failed: 0
Generating Mel-Spectrograms...


In [4]:
# --- 2. Split Data ---
X_train, X_val, X_test, y_train, y_val, y_test, df_train, df_val, df_test = mu.split_dataset(X, y, speakers, df)

print(f"Train shape: {X_train.shape}")
print(f"Val shape:   {X_val.shape}")
print(f"Test shape:  {X_test.shape}")

Train shape: (7859, 128, 130, 1)
Val shape:   (1681, 128, 130, 1)
Test shape:  (1685, 128, 130, 1)


In [5]:
# --- 3. Define Experiments ---
# List of tuples: (Experiment Name, Optimizer, Dropout, Learning Rate)
experiments = [
    ("Baseline_Adam", "adam", 0.0, 0.001),
    ("Baseline_SGD",  "sgd",  0.0, 0.01),
    ("Dropout_0.2",   "adam", 0.2, 0.001),
    ("Low_LR_Adam",   "adam", 0.0, 0.0001),
]

results_summary = []
# Create a directory for results if it doesn't exist
os.makedirs("results", exist_ok=True)

In [6]:
# --- 4. Run Experiments Loop ---
for exp_name, opt, drop, lr in experiments:
    print(f"\n" + "="*40)
    print(f"Running Experiment: {exp_name}")
    print(f"Params: Opt={opt}, Dropout={drop}, LR={lr}")
    print("="*40)

    # Build model using helper function
    model = mu.build_model(
        input_shape=X_train.shape[1:],
        optimizer_name=opt,
        dropout_rate=drop,
        learning_rate=lr
    )

    # Callbacks: Early Stopping and CSV Logger
    log_path = f"results/logs_{exp_name}.csv"
    callbacks = [
        tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True, verbose=1),
        tf.keras.callbacks.CSVLogger(log_path)
    ]

    # Train model
    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=20,
        batch_size=32,
        callbacks=callbacks,
        verbose=1
    )

    # Evaluate on Test Set
    loss, acc = model.evaluate(X_test, y_test, verbose=0)
    print(f"Test Accuracy: {acc:.4f}")

    # Record results
    results_summary.append({
        "Experiment": exp_name,
        "Test_Accuracy": acc,
        "Test_Loss": loss,
        "Optimizer": opt,
        "Dropout": drop,
        "Learning_Rate": lr
    })

    # Save Model
    model_save_path = f"results/model_{exp_name}.keras"
    model.save(model_save_path)
    print(f"Model saved to {model_save_path}")

    # --- IMPORTANT FOR EDA ---
    # Save predictions with metadata for Error Analysis
    print("Generating predictions for analysis...")
    preds_prob = model.predict(X_test, verbose=0).flatten()

    # Create DataFrame with true labels, predictions, AND metadata (filenames)
    # We use df_test which we got from the split function
    pred_df = df_test.copy()
    pred_df["true_label"] = y_test
    pred_df["pred_prob"] = preds_prob
    pred_df["pred_label"] = (preds_prob > 0.5).astype(int)

    # Save to CSV
    pred_df.to_csv(f"results/predictions_{exp_name}.csv", index=False)


Running Experiment: Baseline_Adam
Params: Opt=adam, Dropout=0.0, LR=0.001
Epoch 1/20
246/246 ━━━━━━━━━━━━━━━━━━━━ 33s 130ms/step - accuracy: 0.6549 - loss: 1.3629 - val_accuracy: 0.7198 - val_loss: 0.5518
Epoch 2/20
246/246 ━━━━━━━━━━━━━━━━━━━━ 33s 135ms/step - accuracy: 0.7486 - loss: 0.4963 - val_accuracy: 0.8251 - val_loss: 0.4108
Epoch 3/20
246/246 ━━━━━━━━━━━━━━━━━━━━ 35s 143ms/step - accuracy: 0.8215 - loss: 0.3878 - val_accuracy: 0.7930 - val_loss: 0.4207
Epoch 4/20
246/246 ━━━━━━━━━━━━━━━━━━━━ 31s 128ms/step - accuracy: 0.8633 - loss: 0.3173 - val_accuracy: 0.7579 - val_loss: 0.5195
Epoch 5/20
246/246 ━━━━━━━━━━━━━━━━━━━━ 34s 137ms/step - accuracy: 0.9094 - loss: 0.2200 - val_accuracy: 0.9244 - val_loss: 0.1929
Epoch 6/20
246/246 ━━━━━━━━━━━━━━━━━━━━ 33s 133ms/step - accuracy: 0.9338 - loss: 0.1649 - val_accuracy: 0.9369 - val_loss: 0.1635
Epoch 7/20
246/246 ━━━━━━━━━━━━━━━━━━━━ 32s 131ms/step - accuracy: 0.9018 - loss: 0.2429 - val_accuracy: 0.7311 - val_loss: 0.4998
Epoch 8/

In [8]:
# --- 5. Summary ---
summary_df = pd.DataFrame(results_summary)

# Add a timestamp column with the current date and time
summary_df['timestamp'] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

print("\n=== Final Experiments Summary ===")
print(summary_df)

summary_path = "results/experiments_summary.csv"

# Check if file exists to determine whether to write the header
# Note: If an old file exists without a 'timestamp' column, appending might cause
# structure issues. It is best to delete the old file or manually add a column header to it.
write_header = not os.path.exists(summary_path)

# Save to CSV with mode='a' (append)
summary_df.to_csv(summary_path, mode='a', header=write_header, index=False)

print("\nAll experiments completed.")


=== Final Experiments Summary ===
      Experiment  Test_Accuracy  Test_Loss Optimizer  Dropout  Learning_Rate  \
0  Baseline_Adam       0.934125   0.189988      adam      0.0         0.0010   
1   Baseline_SGD       0.535312   0.690691       sgd      0.0         0.0100   
2    Dropout_0.2       0.920475   0.202343      adam      0.2         0.0010   
3    Low_LR_Adam       0.920475   0.196554      adam      0.0         0.0001   

             timestamp  
0  2025-12-14 21:18:41  
1  2025-12-14 21:18:41  
2  2025-12-14 21:18:41  
3  2025-12-14 21:18:41  

All experiments completed.
